# DSPy - **D**eclarative **S**elf-improving **Py**thon

## Wprowadzenie do DSPy

DSPy to framework do tworzenia, optymalizacji i wdrażania złożonych aplikacji wykorzystujących LLM (Large Language Models). W przeciwieństwie do tradycyjnych podejść opartych na ręcznym tworzeniu promptów, DSPy pozwala na **programowanie** zachowań modeli językowych w sposób systematyczny i modułowy.

## Importy

In [1]:
# Import bibliotek
import dspy
import os

In [2]:
# Konfiguracja klucza API
api_key = os.getenv('OPENAI_API_KEY')

# Konfiguracja modelu nauczyciela (GPT-4.1)
gpt4_1_teacher = dspy.LM(
    model="openai/gpt-4.1",
    api_key=api_key,
    temperature=1.0,
    max_tokens=32000
)

# Konfiguracja modelu docelowego (GPT-4.1-nano)
gpt4_1_nano = dspy.LM(
    model="openai/gpt-4.1-nano",
    api_key=api_key,
    temperature=0.0,
    max_tokens=32000
)

# Główna konfiguracja DSPy
dspy.configure(
    lm=gpt4_1_teacher,       # Domyślny model dla generacji instrukcji
    task_lm=gpt4_1_nano,     # Model dla wykonywania zadań
    cache=False              # Wyłączenie cache dla dynamicznych wyników
)

## Podstawowe koncepcje DSPy

DSPy opiera się na kilku kluczowych koncepcjach:

1. **Moduły (Modules)** - komponenty definiujące zadania dla LLM
2. **Sygnatura (Signature)** - definicja wejść i wyjść modułów
3. **Teleprompter** - mechanizm optymalizacji promptów
4. **Program** - połączenie modułów w potok przetwarzania

Poniżej przedstawimy praktyczne przykłady wykorzystania tych koncepcji.

## Przykład 1: Podstawowe moduły DSPy - Klasyfikacja sentymentu

In [3]:
# Definiujemy sygnaturę dla klasyfikatora sentymentu
class SentimentClassifier(dspy.Signature):
    """Określ sentyment tekstu jako pozytywny, negatywny lub neutralny."""
    
    text = dspy.InputField()
    sentiment = dspy.OutputField(desc="Sentyment tekstu: pozytywny, negatywny lub neutralny")
    explanation = dspy.OutputField(desc="Krótkie wyjaśnienie przypisanego sentymentu")

# Tworzymy moduł bazujący na zdefiniowanej sygnaturze
sentiment_predictor = dspy.Predict(SentimentClassifier)

# Przykładowe teksty do analizy
texts = [
    "Dzisiaj jest piękny dzień, świeci słońce i jestem szczęśliwy.",
    "Niestety, mój pociąg się spóźnił i spóźniłem się na ważne spotkanie.",
    "W Polsce jest 16 województw."
]

# Analiza sentymentu dla każdego tekstu
for text in texts:
    result = sentiment_predictor(text=text)
    print(f"Tekst: {text}")
    print(f"Sentyment: {result.sentiment}")
    print(f"Wyjaśnienie: {result.explanation}")
    print("-" * 80)

Tekst: Dzisiaj jest piękny dzień, świeci słońce i jestem szczęśliwy.
Sentyment: pozytywny
Wyjaśnienie: Tekst wyraża radość i zadowolenie, opisuje ładną pogodę oraz szczęśliwy nastrój autora.
--------------------------------------------------------------------------------
Tekst: Niestety, mój pociąg się spóźnił i spóźniłem się na ważne spotkanie.
Sentyment: negatywny
Wyjaśnienie: Tekst opisuje nieprzyjemną sytuację – spóźnienie pociągu, które skutkowało spóźnieniem na ważne spotkanie. Wskazuje to na negatywne emocje i rozczarowanie.
--------------------------------------------------------------------------------
Tekst: W Polsce jest 16 województw.
Sentyment: neutralny
Wyjaśnienie: Tekst zawiera jedynie obiektywną informację geograficzną bez wyrażania emocji ani opinii.
--------------------------------------------------------------------------------


In [4]:
# wyświetlmy przykładowy prompt
dspy.inspect_history(n=1)





[2025-05-03T22:29:31.999841]

System message:

Your input fields are:
1. `text` (str)
Your output fields are:
1. `sentiment` (str): Sentyment tekstu: pozytywny, negatywny lub neutralny
2. `explanation` (str): Krótkie wyjaśnienie przypisanego sentymentu
All interactions will be structured in the following way, with the appropriate values filled in.

[[ ## text ## ]]
{text}

[[ ## sentiment ## ]]
{sentiment}

[[ ## explanation ## ]]
{explanation}

[[ ## completed ## ]]
In adhering to this structure, your objective is: 
        Określ sentyment tekstu jako pozytywny, negatywny lub neutralny.


User message:

[[ ## text ## ]]
W Polsce jest 16 województw.

Respond with the corresponding output fields, starting with the field `[[ ## sentiment ## ]]`, then `[[ ## explanation ## ]]`, and then ending with the marker for `[[ ## completed ## ]]`.


Response:

[[ ## sentiment ## ]]
neutralny

[[ ## explanation ## ]]
Tekst zawiera jedynie obiektywną informację geograficzną bez wyrażania emocji 

## Przykład 2: Łańcuchy modułów - Analiza i streszczenie artykułów

In [5]:
# Definiujemy sygnaturę dla analizy artykułu
class ArticleAnalyzer(dspy.Signature):
    """Analizuj artykuł i wyodrębnij kluczowe informacje."""
    
    article = dspy.InputField()
    topics = dspy.OutputField(desc="Lista 3-5 głównych tematów poruszanych w artykule")
    key_points = dspy.OutputField(desc="Lista 3-5 najważniejszych punktów z artykułu")

# Definiujemy sygnaturę dla generowania podsumowania
class ArticleSummarizer(dspy.Signature):
    """Stwórz zwięzłe podsumowanie artykułu na podstawie analizy."""
    
    article = dspy.InputField()
    topics = dspy.InputField()
    key_points = dspy.InputField()
    summary = dspy.OutputField(desc="Zwięzłe podsumowanie artykułu w 2-3 zdaniach")

# Tworzymy program łączący analizę i podsumowanie
class ArticleProcessor(dspy.Module):
    def __init__(self):
        super().__init__()
        self.analyzer = dspy.Predict(ArticleAnalyzer)
        self.summarizer = dspy.Predict(ArticleSummarizer)
    
    def forward(self, article):
        # Analiza artykułu
        analysis = self.analyzer(article=article)
        
        # Generowanie podsumowania na podstawie analizy
        summary = self.summarizer(
            article=article,
            topics=analysis.topics,
            key_points=analysis.key_points
        )
        
        return dspy.Prediction(
            topics=analysis.topics,
            key_points=analysis.key_points,
            summary=summary.summary
        )

# Przykładowy artykuł
article = """
Sztuczna inteligencja (AI) w ostatnich latach dokonała ogromnego postępu, szczególnie w dziedzinie modeli językowych. 
Modele takie jak GPT-4.1, Claude 3.7 czy LLaMA 4 potrafią generować tekst, tłumaczyć języki, pisać różne rodzaje kreatywnych 
treści i odpowiadać na pytania w sposób, który wydaje się niemal ludzki. Te osiągnięcia są wynikiem rozwoju architektury 
Transformer oraz zwiększenia ilości danych i mocy obliczeniowej dostępnej do treningu modeli. Jednocześnie, 
wraz z rosnącymi możliwościami AI, pojawiają się nowe wyzwania etyczne i społeczne związane z jej wykorzystaniem. 
Kwestie prywatności danych, bezpieczeństwa, potencjalnej stronniczości algorytmów oraz wpływu automatyzacji na rynek pracy 
stają się coraz bardziej istotne w dyskusjach o przyszłości technologii AI. Rządy i organizacje na całym świecie 
zaczynają wprowadzać regulacje mające na celu zapewnienie odpowiedzialnego rozwoju i wdrażania sztucznej inteligencji.

Historia rozwoju sztucznej inteligencji sięga połowy XX wieku, kiedy to powstały pierwsze koncepcje teoretyczne i proste 
programy AI. Jednak dopiero w ostatniej dekadzie nastąpił przełom związany z uczeniem głębokim i sieciami neuronowymi. 
Kluczowym momentem był rok 2017, kiedy to opublikowano pracę "Attention is All You Need", która wprowadzała architekturę 
Transformer, będącą podstawą dzisiejszych dużych modeli językowych (LLM). Kolejne lata przyniosły modele takie jak BERT, 
GPT-2 i GPT-3. Pojawiły się też systemy text-to-image jak DALL-E czy Midjourney, które potrafią generować obrazy na podstawie 
opisów tekstowych. W 2023 roku upowszechnienie ChatGPT spowodowało masową adopcję technologii AI wśród zwykłych użytkowników, 
otwierając nową erę w interakcji człowiek-komputer.

Zastosowania sztucznej inteligencji obejmują obecnie niemal każdą dziedzinę życia i gospodarki. W medycynie AI wspomaga 
diagnostykę, analizując obrazy medyczne i przewidując potencjalne schorzenia z dokładnością często przewyższającą ludzkich 
specjalistów. W przemyśle systemy AI optymalizują procesy produkcyjne, redukując koszty i zwiększając wydajność. W edukacji 
personalizują doświadczenia uczenia się, dostosowując materiały do indywidualnych potrzeb uczniów. W finansach algorytmy AI 
wykrywają oszustwa, oceniają ryzyko kredytowe i dokonują transakcji giełdowych w milisekundach. Technologie rozpoznawania 
mowy i obrazu rewolucjonizują interfejsy użytkownika, podczas gdy autonomiczne pojazdy zmieniają transport i logistykę. 
Jednocześnie branża kreatywna wykorzystuje AI do generowania muzyki, sztuki i treści literackich, otwierając nowe możliwości 
ekspresji artystycznej, ale też stawiając pytania o granice kreatywności i oryginalności.
"""

# Przetwarzanie artykułu
processor = ArticleProcessor()
result = processor(article)

print("Tematy:")
print(result.topics)
print("\nKluczowe punkty:")
print(result.key_points)
print("\nPodsumowanie:")
print(result.summary)

Tematy:
1. Postęp i możliwości współczesnych modeli językowych AI
2. Historia i rozwój sztucznej inteligencji, w tym architektura Transformer
3. Etyczne i społeczne wyzwania oraz regulacje dotyczące AI
4. Zastosowania sztucznej inteligencji w różnych dziedzinach życia i gospodarki

Kluczowe punkty:
1. Nowoczesne modele językowe AI, takie jak GPT-4.1 czy LLaMA 4, potrafią generować zaawansowane teksty i odpowiadać na pytania na poziomie zbliżonym do ludzkiego.
2. Przełomem w rozwoju AI była publikacja architektury Transformer w 2017 roku, która umożliwiła stworzenie dużych, skutecznych modeli językowych.
3. Z szerokim zastosowaniem AI wiążą się poważne wyzwania, takie jak kwestie prywatności, bezpieczeństwa, stronniczości oraz wpływu na rynek pracy, co prowadzi do tworzenia nowych regulacji.
4. Sztuczna inteligencja znajduje zastosowanie m.in. w medycynie, przemyśle, edukacji, finansach, technologii rozpoznawania obrazu i dźwięku, transporcie oraz branży kreatywnej.

Podsumowanie:
Artyk

In [7]:
# wyświetlmy przykładowy prompt
dspy.inspect_history(n=2)





[2025-05-03T22:31:24.366290]

System message:

Your input fields are:
1. `article` (str)
Your output fields are:
1. `topics` (str): Lista 3-5 głównych tematów poruszanych w artykule
2. `key_points` (str): Lista 3-5 najważniejszych punktów z artykułu
All interactions will be structured in the following way, with the appropriate values filled in.

[[ ## article ## ]]
{article}

[[ ## topics ## ]]
{topics}

[[ ## key_points ## ]]
{key_points}

[[ ## completed ## ]]
In adhering to this structure, your objective is: 
        Analizuj artykuł i wyodrębnij kluczowe informacje.


User message:

[[ ## article ## ]]

Sztuczna inteligencja (AI) w ostatnich latach dokonała ogromnego postępu, szczególnie w dziedzinie modeli językowych. 
Modele takie jak GPT-4.1, Claude 3.7 czy LLaMA 4 potrafią generować tekst, tłumaczyć języki, pisać różne rodzaje kreatywnych 
treści i odpowiadać na pytania w sposób, który wydaje się niemal ludzki. Te osiągnięcia są wynikiem rozwoju architektury 
Transformer or

## Przykład 3: Zastosowanie DSPy w RAGu

In [8]:
# Definiujemy sygnaturę dla generowania odpowiedzi na pytania
class RAG(dspy.Signature):
    """Odpowiadaj na pytania na podstawie dostępnych informacji."""
    
    context = dspy.InputField(desc="Kontekst zawierający informacje")
    question = dspy.InputField()
    answer = dspy.OutputField(desc="Odpowiedź na pytanie oparta wyłącznie na dostarczonym kontekście")

# Tworzymy moduł QA
qa_model = dspy.Predict(RAG)

# Przykładowy kontekst i pytanie
context = """
Uniwersytet Warszawski (UW) – polska publiczna uczelnia wyższa założona 19 listopada 1816 roku w Warszawie. 
Największy uniwersytet w Polsce, od lat zajmujący pierwsze miejsce w krajowych rankingach szkół wyższych. 
W 2016 roku Uniwersytet Warszawski obchodził jubileusz 200-lecia istnienia. 
W skład uczelni wchodzi 21 wydziałów, 30 innych jednostek dydaktycznych oraz ponad 100 kierunków studiów. 
W roku akademickim 2018/2019 zatrudnionych było 3,8 tys. nauczycieli akademickich, a kształciło się 42,5 tys. studentów, 
3 tys. doktorantów i 2,5 tys. słuchaczy studiów podyplomowych.
"""

questions = [
    "Kiedy założono Uniwersytet Warszawski?",
    "Ile wydziałów ma UW?",
    "Ilu studentów kształciło się na UW w roku 2018/2019?"
]

for question in questions:
    answer = qa_model(context=context, question=question)
    print(f"Pytanie: {question}")
    print(f"Odpowiedź: {answer.answer}")
    print("-" * 80)

Pytanie: Kiedy założono Uniwersytet Warszawski?
Odpowiedź: Uniwersytet Warszawski założono 19 listopada 1816 roku.
--------------------------------------------------------------------------------
Pytanie: Ile wydziałów ma UW?
Odpowiedź: Uniwersytet Warszawski ma 21 wydziałów.
--------------------------------------------------------------------------------
Pytanie: Ilu studentów kształciło się na UW w roku 2018/2019?
Odpowiedź: W roku akademickim 2018/2019 na Uniwersytecie Warszawskim kształciło się 42,5 tys. studentów.
--------------------------------------------------------------------------------


In [9]:
# wyświetlmy przykładowy prompt
dspy.inspect_history(n=1)





[2025-05-03T22:35:25.411589]

System message:

Your input fields are:
1. `context` (str): Kontekst zawierający informacje
2. `question` (str)
Your output fields are:
1. `answer` (str): Odpowiedź na pytanie oparta wyłącznie na dostarczonym kontekście
All interactions will be structured in the following way, with the appropriate values filled in.

[[ ## context ## ]]
{context}

[[ ## question ## ]]
{question}

[[ ## answer ## ]]
{answer}

[[ ## completed ## ]]
In adhering to this structure, your objective is: 
        Odpowiadaj na pytania na podstawie dostępnych informacji.


User message:

[[ ## context ## ]]

Uniwersytet Warszawski (UW) – polska publiczna uczelnia wyższa założona 19 listopada 1816 roku w Warszawie. 
Największy uniwersytet w Polsce, od lat zajmujący pierwsze miejsce w krajowych rankingach szkół wyższych. 
W 2016 roku Uniwersytet Warszawski obchodził jubileusz 200-lecia istnienia. 
W skład uczelni wchodzi 21 wydziałów, 30 innych jednostek dydaktycznych oraz ponad 10

## Przykład 4: Chain-of-Thought z DSPy

In [14]:
# Definiujemy sygnaturę dla rozumowania krok po kroku
class MathProblemSolver(dspy.Signature):
    """Rozwiązuj problemy matematyczne krok po kroku."""
    
    problem = dspy.InputField(desc="Problem obliczeniowy do rozwiązania")
    reasoning = dspy.OutputField(desc="Szczegółowe rozumowanie krok po kroku")
    answer = dspy.OutputField(desc="Końcowa odpowiedź na problem. Format to liczba z jednostką (w przypadku jednostek odmienialnych musi być podana w formie poprawnej gramatycznie, np. '2 kg jabłek', '5%' lub '8 zł').")

# Tworzymy moduł CoT (Chain of Thought)
math_solver = dspy.ChainOfThought(MathProblemSolver)

# Przykładowy problem matematyczny
problem = """
W sklepie spożywczym jabłka kosztują 3,50 zł za kilogram, a gruszki 4,75 zł za kilogram. 
Pani Kowalska kupiła 2,5 kg jabłek i 1,8 kg gruszek. Ile zapłaciła za wszystkie owoce?
"""

# Rozwiązanie problemu
solution = math_solver(problem=problem)

print("Problem:")
print(problem)
print("\nRozumowanie:")
print(solution.reasoning)
print("\nOdpowiedź:")
print(solution.answer)

Problem:

W sklepie spożywczym jabłka kosztują 3,50 zł za kilogram, a gruszki 4,75 zł za kilogram. 
Pani Kowalska kupiła 2,5 kg jabłek i 1,8 kg gruszek. Ile zapłaciła za wszystkie owoce?


Rozumowanie:
Najpierw obliczam koszt zakupu jabłek: 2,5 kg × 3,50 zł/kg = 8,75 zł.  
Następnie obliczam koszt zakupu gruszek: 1,8 kg × 4,75 zł/kg = 8,55 zł.  
Teraz sumuję te dwie kwoty, aby uzyskać łączny koszt zakupu wszystkich owoców: 8,75 zł + 8,55 zł = 17,30 zł.

Odpowiedź:
17,30 zł


## Przykład 5: Optymalizacja promptów

Optymalizujemy program MathProblemSolver z poprzedniego przykładu

In [15]:
# Przygotowanie zestawu treningowego
train_data = [
    dspy.Example(
        problem="Jeśli 5 koszy zawiera po 8 jabłek każdy, to ile jest wszystkich jabłek?",
        reasoning="Aby obliczyć całkowitą liczbę jabłek, mnożę liczbę koszy przez liczbę jabłek w każdym koszu: 5 koszy × 8 jabłek = 40 jabłek.",
        answer="40 jabłek"
    ).with_inputs("problem"),
    dspy.Example(
        problem="Bilet do kina kosztuje 22 zł dla osoby dorosłej i 14 zł dla dziecka. Ile zapłaci rodzina składająca się z 2 dorosłych i 3 dzieci?",
        reasoning="Obliczam koszt dla dorosłych: 2 × 22 zł = 44 zł. Obliczam koszt dla dzieci: 3 × 14 zł = 42 zł. Całkowity koszt to: 44 zł + 42 zł = 86 zł.",
        answer="86 zł"
    ).with_inputs("problem"),
    dspy.Example(
        problem="Samochód przejechał 240 km w 3 godziny. Jaka była jego średnia prędkość?",
        reasoning="Aby obliczyć średnią prędkość, dzielę przebytą odległość przez czas: 240 km ÷ 3 h = 80 km/h.",
        answer="80 km/h"
    ).with_inputs("problem"),
    dspy.Example(
        problem="Na przyjęciu było 12 osób. Każda osoba zjadła 3 kawałki tortu. Ile kawałków tortu zjedzono łącznie?",
        reasoning="Obliczam całkowitą liczbę kawałków tortu: 12 osób × 3 kawałki = 36 kawałków.",
        answer="36 kawałków"
    ).with_inputs("problem"),
    dspy.Example(
        problem="Na placu zabaw bawi się 15 dzieci. Jeśli 5 z nich poszło do domu, ile dzieci zostało na placu zabaw?",
        reasoning="Obliczam liczbę dzieci, które zostały: 15 dzieci - 5 dzieci = 10 dzieci.",
        answer="10 dzieci"
    ).with_inputs("problem"),
    dspy.Example(
        problem="W klasie jest 20 uczniów. Jeśli 4 uczniów nie przyszło do szkoły, ilu uczniów jest w klasie?",
        reasoning="Obliczam liczbę uczniów w klasie: 20 uczniów - 4 uczniów = 16 uczniów.",
        answer="16 uczniów"
    ).with_inputs("problem"),
    dspy.Example(
        problem="Na stole leży 10 jabłek. Jeśli 3 z nich są zielone, a reszta czerwona, ile jabłek jest czerwonych?",
        reasoning="Obliczam liczbę czerwonych jabłek: 10 jabłek - 3 zielone = 7 czerwonych.",
        answer="7 czerwonych jabłek"
    ).with_inputs("problem"),
    dspy.Example(
        problem="W parku rośnie 8 drzew. Jeśli każde drzewo ma 5 gałęzi, ile gałęzi jest łącznie?",
        reasoning="Obliczam całkowitą liczbę gałęzi: 8 drzew × 5 gałęzi = 40 gałęzi.",
        answer="40 gałęzi"
    ).with_inputs("problem"),
    dspy.Example(
        problem="Na boisku gra 11 zawodników w jednej drużynie. Ile zawodników gra w dwóch drużynach?",
        reasoning="Obliczam całkowitą liczbę zawodników: 11 zawodników × 2 drużyny = 22 zawodników.",
        answer="22 zawodników"
    ).with_inputs("problem"),
    dspy.Example(
        problem="W pudełku jest 24 cukierków. Jeśli podzielę je równo między 6 dzieci, ile cukierków dostanie każde dziecko?",
        reasoning="Obliczam liczbę cukierków na dziecko: 24 cukierków ÷ 6 dzieci = 4 cukierków.",
        answer="4 cukierki"
    ).with_inputs("problem"),
    dspy.Example(
        problem="Siła 150 N działa na powierzchnię 3 m². Jakie jest ciśnienie?",
        reasoning="Aby obliczyć ciśnienie, dzielę siłę przez powierzchnię: 150 N ÷ 3 m² = 50 N/m².",
        answer="50 N/m²"
    ).with_inputs("problem"),
    dspy.Example(
        problem="Ciało o masie 5 kg porusza się z przyspieszeniem 2 m/s². Jaka siła działa na to ciało?",
        reasoning="Korzystam z drugiej zasady dynamiki Newtona: F = m × a. F = 5 kg × 2 m/s² = 10 N.",
        answer="10 N"
    ).with_inputs("problem"),
    dspy.Example(
        problem="Samochód poruszający się z prędkością 20 m/s zahamował do zera w ciągu 4 sekund. Jakie było opóźnienie?",
        reasoning="Opóźnienie to zmiana prędkości w czasie: a = Δv ÷ t = (0 - 20 m/s) ÷ 4 s = -20 m/s ÷ 4 s = -5 m/s². Wartość bezwzględna opóźnienia wynosi 5 m/s².",
        answer="5 m/s²"
    ).with_inputs("problem"),
    dspy.Example(
        problem="Skrzynia o masie 200 kg jest podnoszona na wysokość 5 metrów. Jaką pracę wykonano przeciwko sile grawitacji?",
        reasoning="Praca to siła × droga. W tym przypadku siła to ciężar skrzyni: F = m × g = 200 kg × 9,8 m/s² = 1960 N. Praca: W = F × h = 1960 N × 5 m = 9800 J.",
        answer="9800 J"
    ).with_inputs("problem"),
    dspy.Example(
        problem="Samochód o masie 1000 kg porusza się z prędkością 72 km/h. Jaką energię kinetyczną posiada?",
        reasoning="Najpierw zamieniam jednostki: 72 km/h = 20 m/s. Energia kinetyczna to: Ek = (m × v²) ÷ 2 = (1000 kg × (20 m/s)²) ÷ 2 = (1000 kg × 400 m²/s²) ÷ 2 = 200000 J = 200 kJ.",
        answer="200 kJ"
    ).with_inputs("problem"),
    dspy.Example(
        problem="Jeśli 1 metr sześcienny wody waży 1000 kg, ile waży 5 litrów wody?",
        reasoning="1 m³ = 1000 litrów, więc 1 litr waży 1 kg. Zatem 5 litrów waży 5 kg.",
        answer="5 kg"
    ).with_inputs("problem"),
    dspy.Example(
        problem="Droga z Warszawy do Krakowa wynosi 290 km. Jeśli samochód spala 7 litrów benzyny na 100 km, ile benzyny zużyje na przejazd w obie strony?",
        reasoning="Droga w obie strony to: 290 km × 2 = 580 km. Zużycie benzyny: 580 km × (7 l ÷ 100 km) = 580 km × 0,07 l/km = 40,6 l.",
        answer="40,6 l"
    ).with_inputs("problem"),
    dspy.Example(
        problem="Ciało o temperaturze 80°C stygnie do temperatury 20°C. O ile stopni obniżyła się temperatura ciała?",
        reasoning="Zmiana temperatury to różnica temperatur końcowej i początkowej: ΔT = T_końcowa - T_początkowa = 20°C - 80°C = -60°C. Wartość bezwzględna zmiany to 60°C.",
        answer="60°C"
    ).with_inputs("problem"),
    dspy.Example(
        problem="Jeśli 1 kWh energii kosztuje 0,65 zł, ile kosztuje działanie urządzenia o mocy 2000 W przez 5 godzin?",
        reasoning="Najpierw obliczam zużytą energię: E = P × t = 2000 W × 5 h = 10000 Wh = 10 kWh. Koszt: 10 kWh × 0,65 zł/kWh = 6,50 zł.",
        answer="6,50 zł"
    ).with_inputs("problem"),
    dspy.Example(
        problem="W sklepie kupiłem 2,5 kg ziemniaków po 2,20 zł za kilogram, 1,5 kg cebuli po 3,40 zł za kilogram i 0,75 kg marchwi po 2,80 zł za kilogram. Ile zapłaciłem łącznie?",
        reasoning="Koszt ziemniaków: 2,5 kg × 2,20 zł/kg = 5,50 zł. Koszt cebuli: 1,5 kg × 3,40 zł/kg = 5,10 zł. Koszt marchwi: 0,75 kg × 2,80 zł/kg = 2,10 zł. Łączny koszt: 5,50 zł + 5,10 zł + 2,10 zł = 12,70 zł.",
        answer="12,70 zł"
    ).with_inputs("problem"),
    dspy.Example(
        problem="W zbiorniku o pojemności 50 litrów jest 30 litrów wody. Dolewamy jeszcze 15 litrów. Jaki procent zbiornika jest pełny?",
        reasoning="Łączna ilość wody: 30 l + 15 l = 45 l. Procent wypełnienia: (45 l ÷ 50 l) × 100% = 0,9 × 100% = 90%.",
        answer="90%"
    ).with_inputs("problem"),
    dspy.Example(
        problem="Jeśli 30% uczniów w klasie to chłopcy, a w klasie jest 12 chłopców, ilu uczniów liczy cała klasa?",
        reasoning="Oznaczmy liczbę wszystkich uczniów jako x. Wiemy, że 30% z x to 12, więc: 0,3x = 12. Stąd: x = 12 ÷ 0,3 = 40.",
        answer="40 uczniów"
    ).with_inputs("problem"),
    dspy.Example(
        problem="Cenę produktu podwyższono o 20%, a później obniżono o 10%. Jaką część pierwotnej ceny stanowi końcowa cena?",
        reasoning="Jeśli początkowa cena to x, to po podwyżce będzie: 1,2x. Po obniżce o 10%: 1,2x × 0,9 = 1,08x. Końcowa cena stanowi 108% pierwotnej ceny.",
        answer="108%"
    ).with_inputs("problem"),
    dspy.Example(
        problem="Pole prostokąta wynosi 24 cm². Jeśli jego szerokość to 4 cm, jaka jest długość?",
        reasoning="Pole prostokąta to iloczyn długości i szerokości: P = a × b. Znamy pole P = 24 cm² i szerokość b = 4 cm. Wyznaczam długość: a = P ÷ b = 24 cm² ÷ 4 cm = 6 cm.",
        answer="6 cm"
    ).with_inputs("problem"),
    dspy.Example(
        problem="Promień okręgu wynosi 5 cm. Jakie jest pole koła?",
        reasoning="Pole koła obliczam ze wzoru: P = π × r². P = 3,14 × (5 cm)² = 3,14 × 25 cm² = 78,5 cm².",
        answer="78,5 cm²"
    ).with_inputs("problem"),
    dspy.Example(
        problem="Jeśli sześcian ma krawędź długości 3 cm, jaka jest jego objętość?",
        reasoning="Objętość sześcianu obliczam ze wzoru: V = a³, gdzie a to długość krawędzi. V = (3 cm)³ = 27 cm³.",
        answer="27 cm³"
    ).with_inputs("problem"),
    dspy.Example(
        problem="Jeśli 1 cal to 2,54 cm, to ile centymetrów ma przedmiot o długości 10 cali?",
        reasoning="Zamieniam cale na centymetry: 10 cali × 2,54 cm/cal = 25,4 cm.",
        answer="25,4 cm"
    ).with_inputs("problem"),
    dspy.Example(
        problem="Jeśli pH soku cytrynowego wynosi 2, a pH krwi wynosi 7,4, ile razy bardziej kwasowy jest sok cytrynowy od krwi?",
        reasoning="Skala pH jest logarytmiczna, więc różnica o 1 oznacza 10-krotną różnicę kwasowości. Różnica pH wynosi: 7,4 - 2 = 5,4. Zatem sok cytrynowy jest 10^5,4 ≈ 251189 razy bardziej kwasowy niż krew.",
        answer="251189 razy"
    ).with_inputs("problem"),
    dspy.Example(
        problem="W szkole jest 600 uczniów, z czego 40% to dziewczynki. Ile jest chłopców?",
        reasoning="Liczba dziewcząt: 600 uczniów × 0,4 = 240 dziewcząt. Liczba chłopców: 600 uczniów - 240 dziewcząt = 360 chłopców.",
        answer="360 chłopców"
    ).with_inputs("problem"),
    dspy.Example(
        problem="Jeśli 3 pracowników może wykonać zadanie w 12 godzin, to ile czasu potrzeba, aby 4 pracowników wykonało to samo zadanie?",
        reasoning="Zakładając, że wszyscy pracownicy pracują z taką samą wydajnością, całkowita ilość pracy to: 3 pracowników × 12 godzin = 36 roboczogodzin. Czas potrzebny dla 4 pracowników: 36 roboczogodzin ÷ 4 pracowników = 9 godzin.",
        answer="9 godzin"
    ).with_inputs("problem"),
    dspy.Example(
        problem="Jeśli dystans między miastami na mapie wynosi 6 cm, a skala mapy to 1:100000, jaka jest rzeczywista odległość między tymi miastami w kilometrach?",
        reasoning="Rzeczywista odległość: 6 cm × 100000 = 600000 cm = 6000 m = 6 km.",
        answer="6 km"
    ).with_inputs("problem"),
    dspy.Example(
        problem="Jeśli 200 g substancji zawiera 80 g składnika A, to jaki jest procentowy udział składnika A?",
        reasoning="Procentowy udział składnika A: (80 g ÷ 200 g) × 100% = 0,4 × 100% = 40%.",
        answer="40%"
    ).with_inputs("problem"),
    dspy.Example(
        problem="Samochód jedzie ze średnią prędkością 60 km/h. Jaką odległość pokona w ciągu 2,5 godziny?",
        reasoning="Odległość to prędkość × czas: s = v × t = 60 km/h × 2,5 h = 150 km.",
        answer="150 km"
    ).with_inputs("problem"),
    dspy.Example(
        problem="Jeśli 1 mol gazu zajmuje objętość 22,4 litra w warunkach normalnych, to jaką objętość zajmują 3 mole tego gazu?",
        reasoning="Objętość 3 moli gazu: 3 mole × 22,4 l/mol = 67,2 l.",
        answer="67,2 l"
    ).with_inputs("problem"),
    dspy.Example(
        problem="Przy wymianie walut kurs euro wynosi 4,50 zł. Ile euro otrzymamy za 900 zł?",
        reasoning="Ilość euro: 900 zł ÷ 4,50 zł/euro = 200 euro.",
        answer="200 euro"
    ).with_inputs("problem"),
    dspy.Example(
        problem="Jeśli autobus wyjeżdża z przystanku co 15 minut, zaczynając od 6:00, to o której godzinie będzie 10-ty autobus?",
        reasoning="10-ty autobus wyjedzie po upływie 9 × 15 minut od godziny 6:00 (pierwszy wyjeżdża o 6:00). 9 × 15 minut = 135 minut = 2 godziny i 15 minut. 6:00 + 2:15 = 8:15.",
        answer="8:15"
    ).with_inputs("problem"),
    dspy.Example(
        problem="W dwóch pudełkach jest łącznie 50 kredek. Jeśli w pierwszym pudełku jest o 10 kredek więcej niż w drugim, to ile kredek jest w pierwszym pudełku?",
        reasoning="Oznaczmy liczbę kredek w drugim pudełku jako x. Wtedy w pierwszym jest x + 10. Razem mają 50 kredek: x + (x + 10) = 50. Stąd: 2x + 10 = 50, 2x = 40, x = 20. W pierwszym pudełku: x + 10 = 20 + 10 = 30 kredek.",
        answer="30 kredek"
    ).with_inputs("problem"),
    dspy.Example(
        problem="Jeśli trójkąt ma pole 24 cm² i wysokość 6 cm, to jaka jest długość podstawy?",
        reasoning="Pole trójkąta to: P = (a × h) ÷ 2, gdzie a to podstawa, a h to wysokość. Przekształcam wzór: a = (2 × P) ÷ h = (2 × 24 cm²) ÷ 6 cm = 48 cm² ÷ 6 cm = 8 cm.",
        answer="8 cm"
    ).with_inputs("problem"),
    dspy.Example(
        problem="Jeśli zmieszamy 300 ml wody o temperaturze 20°C z 200 ml wody o temperaturze 80°C, to jaka będzie temperatura mieszaniny (zakładając brak strat ciepła)?",
        reasoning="Temperatura mieszaniny to średnia ważona temperatur: T = (m₁ × T₁ + m₂ × T₂) ÷ (m₁ + m₂) = (300 ml × 20°C + 200 ml × 80°C) ÷ (300 ml + 200 ml) = (6000 ml·°C + 16000 ml·°C) ÷ 500 ml = 22000 ml·°C ÷ 500 ml = 44°C.",
        answer="44°C"
    ).with_inputs("problem"),
    dspy.Example(
        problem="Jeśli 1 kg mąki kosztuje 2,50 zł, a 1 kg cukru kosztuje 3,00 zł, to ile zapłacimy za 2 kg mąki i 1 kg cukru?",
        reasoning="Koszt mąki: 2 kg × 2,50 zł/kg = 5,00 zł. Koszt cukru: 1 kg × 3,00 zł/kg = 3,00 zł. Łączny koszt: 5,00 zł + 3,00 zł = 8,00 zł.",
        answer="8,00 zł"
    ).with_inputs("problem"),
]


In [39]:
# Definiujemy moduł do optymalizacji
class SimpleMathSolver(dspy.Module):
    def __init__(self):
        super().__init__()
        self.solver = dspy.ChainOfThought(MathProblemSolver)
    
    def forward(self, problem):
        solution = self.solver(problem=problem)
        return solution

def validate_answer(example, pred, trace=None):
    return 1.0 if pred.answer == example.answer else 0.0

# Tworzymy optymalizator ('teleprompter') z wykorzystaniem BootstrapFewShotWithRandomSearch
teleprompter = dspy.teleprompt.BootstrapFewShotWithRandomSearch(
    metric=validate_answer,
    max_bootstrapped_demos=4,
    max_labeled_demos=4
)

# Optymalizujemy moduł na danych treningowych
optimized_solver_demos = teleprompter.compile(
    SimpleMathSolver(),
    trainset=train_data
)

# Testowanie zoptymalizowanego modelu
test_problem = "W sklepie internetowym koszulka kosztuje 79 zł, a dostawa 12 zł. Jeśli kupimy 3 koszulki, to ile zapłacimy łącznie z dostawą?"
result = optimized_solver_demos(problem=test_problem)

print("Problem testowy:")
print(test_problem)
print("\nRozumowanie:")
print(result.reasoning)
print("\nOdpowiedź:")
print(result.answer)

Going to sample between 1 and 4 traces per predictor.
Will attempt to bootstrap 16 candidate sets.
Average Metric: 32.00 / 40 (80.0%): 100%|██████████| 40/40 [00:00<00:00, 2384.14it/s]

2025/05/01 20:27:30 INFO dspy.evaluate.evaluate: Average Metric: 32.0 / 40 (80.0%)



New best score: 80.0 for seed -3
Scores so far: [80.0]
Best score so far: 80.0
Average Metric: 33.00 / 40 (82.5%): 100%|██████████| 40/40 [00:00<00:00, 2869.76it/s]

2025/05/01 20:27:30 INFO dspy.evaluate.evaluate: Average Metric: 33.0 / 40 (82.5%)



New best score: 82.5 for seed -2
Scores so far: [80.0, 82.5]
Best score so far: 82.5


...

  5%|▌         | 2/40 [00:00<00:00, 1549.72it/s]

Bootstrapped 2 full traces after 2 examples for up to 1 rounds, amounting to 2 attempts.


Average Metric: 32.00 / 40 (80.0%): 100%|██████████| 40/40 [00:00<00:00, 2932.67it/s]

2025/05/01 20:27:31 INFO dspy.evaluate.evaluate: Average Metric: 32.0 / 40 (80.0%)



Scores so far: [80.0, 82.5, 77.5, 80.0, 80.0, 77.5, 75.0, 77.5, 77.5, 82.5, 75.0, 82.5, 82.5, 82.5, 82.5, 82.5, 87.5, 82.5, 80.0]
Best score so far: 87.5
19 candidate programs found.
Problem testowy:
W sklepie internetowym koszulka kosztuje 79 zł, a dostawa 12 zł. Jeśli kupimy 3 koszulki, to ile zapłacimy łącznie z dostawą?

Rozumowanie:
Cena za 3 koszulki: 3 × 79 zł = 237 zł. Koszt dostawy wynosi 12 zł. Całkowity koszt to 237 zł + 12 zł = 249 zł.

Odpowiedź:
249 zł


In [40]:
# Sygnatura dla predyktora
optimized_solver_demos.predictors()[0]

Predict(StringSignature(problem -> reasoning, answer
    instructions='Rozwiązuj problemy matematyczne krok po kroku.'
    problem = Field(annotation=str required=True json_schema_extra={'desc': 'Problem obliczeniowy do rozwiązania', '__dspy_field_type': 'input', 'prefix': 'Problem:'})
    reasoning = Field(annotation=str required=True json_schema_extra={'desc': 'Szczegółowe rozumowanie krok po kroku', '__dspy_field_type': 'output', 'prefix': 'Reasoning:'})
    answer = Field(annotation=str required=True json_schema_extra={'desc': "Końcowa odpowiedź na problem. Format to liczba z jednostką (w przypadku jednostek odmienialnych musi być podana w formie poprawnej gramatycznie, np. '2 kg jabłek', '5%' lub '8 zł').", '__dspy_field_type': 'output', 'prefix': 'Answer:'})
))

In [41]:
# Wybrane przykłady few-shot
optimized_solver_demos.predictors()[0].demos

[Example({'augmented': True, 'problem': 'Bilet do kina kosztuje 22 zł dla osoby dorosłej i 14 zł dla dziecka. Ile zapłaci rodzina składająca się z 2 dorosłych i 3 dzieci?', 'reasoning': 'Koszt biletów dla 2 dorosłych: 2 × 22 zł = 44 zł. Koszt biletów dla 3 dzieci: 3 × 14 zł = 42 zł. Łączny koszt to 44 zł + 42 zł = 86 zł.', 'answer': '86 zł'}) (input_keys=None),
 Example({'augmented': True, 'problem': 'W zbiorniku o pojemności 50 litrów jest 30 litrów wody. Dolewamy jeszcze 15 litrów. Jaki procent zbiornika jest pełny?', 'reasoning': 'Łączna ilość wody po dolaniu to 30 litrów + 15 litrów = 45 litrów. Procent zapełnienia zbiornika to (45/50) × 100% = 90%.', 'answer': '90%'}) (input_keys=None),
 Example({'augmented': True, 'problem': 'Jeśli 1 metr sześcienny wody waży 1000 kg, ile waży 5 litrów wody?', 'reasoning': '1 metr sześcienny to 1000 litrów i waży 1000 kg. Zatem 1 litr waży 1 kg. 5 litrów waży 5 × 1 kg = 5 kg.', 'answer': '5 kg'}) (input_keys=None),
 Example({'problem': 'Jeśli 1 m

In [42]:
# wyświetlmy przykładowy prompt
dspy.inspect_history(n=1)





[2025-05-01T20:27:31.225834]

System message:

Your input fields are:
1. `problem` (str): Problem obliczeniowy do rozwiązania
Your output fields are:
1. `reasoning` (str): Szczegółowe rozumowanie krok po kroku
2. `answer` (str): Końcowa odpowiedź na problem. Format to liczba z jednostką (w przypadku jednostek odmienialnych musi być podana w formie poprawnej gramatycznie, np. '2 kg jabłek', '5%' lub '8 zł').
All interactions will be structured in the following way, with the appropriate values filled in.

[[ ## problem ## ]]
{problem}

[[ ## reasoning ## ]]
{reasoning}

[[ ## answer ## ]]
{answer}

[[ ## completed ## ]]
In adhering to this structure, your objective is: 
        Rozwiązuj problemy matematyczne krok po kroku.


User message:

[[ ## problem ## ]]
Bilet do kina kosztuje 22 zł dla osoby dorosłej i 14 zł dla dziecka. Ile zapłaci rodzina składająca się z 2 dorosłych i 3 dzieci?


Assistant message:

[[ ## reasoning ## ]]
Koszt biletów dla 2 dorosłych: 2 × 22 zł = 44 zł. Kos

... i optymalizacja zarówno instrukcji, jak i przykładów few-shot

In [49]:
# Tworzymy optymalizator ('teleprompter') z wykorzystaniem MIPROv2
teleprompter = dspy.teleprompt.MIPROv2(
    metric=validate_answer,
    max_bootstrapped_demos=4,
    max_labeled_demos=4,
    auto='heavy'
)

# Optymalizujemy moduł na danych treningowych
optimized_solver_instruction_demos = teleprompter.compile(
    SimpleMathSolver(),
    trainset=train_data
)

# Testowanie zoptymalizowanego modelu
test_problem = "W sklepie internetowym koszulka kosztuje 79 zł, a dostawa 12 zł. Jeśli kupimy 3 koszulki, to ile zapłacimy łącznie z dostawą?"
result = optimized_solver_instruction_demos(problem=test_problem)

print("Problem testowy:")
print(test_problem)
print("\nRozumowanie:")
print(result.reasoning)
print("\nOdpowiedź:")
print(result.answer)

2025/05/01 20:43:27 INFO dspy.teleprompt.mipro_optimizer_v2: 
RUNNING WITH THE FOLLOWING HEAVY AUTO RUN SETTINGS:
num_trials: 50
minibatch: False
num_candidates: 38
valset size: 32

2025/05/01 20:43:28 INFO dspy.teleprompt.mipro_optimizer_v2: 
==> STEP 1: BOOTSTRAP FEWSHOT EXAMPLES <==
2025/05/01 20:43:28 INFO dspy.teleprompt.mipro_optimizer_v2: These will be used as few-shot example candidates for our program and for creating instructions.

2025/05/01 20:43:28 INFO dspy.teleprompt.mipro_optimizer_v2: Bootstrapping N=38 sets of demonstrations...


Bootstrapping set 1/38
Bootstrapping set 2/38
Bootstrapping set 3/38


 62%|██████▎   | 5/8 [00:00<00:00, 370.21it/s]


Bootstrapped 4 full traces after 5 examples for up to 1 rounds, amounting to 5 attempts.
Bootstrapping set 4/38


 25%|██▌       | 2/8 [00:00<00:00, 364.75it/s]


...


Average Metric: 24.00 / 32 (75.0%): 100%|██████████| 32/32 [00:22<00:00,  1.44it/s]

2025/05/01 21:08:35 INFO dspy.evaluate.evaluate: Average Metric: 24.0 / 32 (75.0%)
2025/05/01 21:08:35 INFO dspy.teleprompt.mipro_optimizer_v2: Score: 75.0 with parameters ['Predictor 0: Instruction 6', 'Predictor 0: Few-Shot Set 37'].
2025/05/01 21:08:35 INFO dspy.teleprompt.mipro_optimizer_v2: Scores so far: [78.12, 78.12, 65.62, 65.62, 65.62, 71.88, 65.62, 65.62, 71.88, 71.88, 81.25, 75.0, 68.75, 78.12, 78.12, 81.25, 71.88, 71.88, 78.12, 65.62, 81.25, 81.25, 78.12, 81.25, 78.12, 68.75, 68.75, 68.75, 65.62, 71.88, 71.88, 68.75, 75.0, 71.88, 75.0, 75.0, 81.25, 78.12, 68.75, 81.25, 78.12, 78.12, 81.25, 78.12, 62.5, 78.12, 75.0, 87.5, 71.88, 75.0]
2025/05/01 21:08:35 INFO dspy.teleprompt.mipro_optimizer_v2: Best score so far: 87.5
2025/05/01 21:08:35 INFO dspy.teleprompt.mipro_optimizer_v2: =========================


2025/05/01 21:08:35 INFO dspy.teleprompt.mipro_optimizer_v2: ===== Trial 51 / 50 =====



Average Metric: 26.00 / 32 (81.2%): 100%|██████████| 32/32 [00:14<00:00,  2.21it/s]

2025/05/01 21:08:49 INFO dspy.evaluate.evaluate: Average Metric: 26.0 / 32 (81.2%)
2025/05/01 21:08:49 INFO dspy.teleprompt.mipro_optimizer_v2: Score: 81.25 with parameters ['Predictor 0: Instruction 19', 'Predictor 0: Few-Shot Set 10'].
2025/05/01 21:08:49 INFO dspy.teleprompt.mipro_optimizer_v2: Scores so far: [78.12, 78.12, 65.62, 65.62, 65.62, 71.88, 65.62, 65.62, 71.88, 71.88, 81.25, 75.0, 68.75, 78.12, 78.12, 81.25, 71.88, 71.88, 78.12, 65.62, 81.25, 81.25, 78.12, 81.25, 78.12, 68.75, 68.75, 68.75, 65.62, 71.88, 71.88, 68.75, 75.0, 71.88, 75.0, 75.0, 81.25, 78.12, 68.75, 81.25, 78.12, 78.12, 81.25, 78.12, 62.5, 78.12, 75.0, 87.5, 71.88, 75.0, 81.25]
2025/05/01 21:08:49 INFO dspy.teleprompt.mipro_optimizer_v2: Best score so far: 87.5
2025/05/01 21:08:49 INFO dspy.teleprompt.mipro_optimizer_v2: =========================


2025/05/01 21:08:49 INFO dspy.teleprompt.mipro_optimizer_v2: Returning best identified program with score 87.5!



Problem testowy:
W sklepie internetowym koszulka kosztuje 79 zł, a dostawa 12 zł. Jeśli kupimy 3 koszulki, to ile zapłacimy łącznie z dostawą?

Rozumowanie:
Najpierw obliczam koszt 3 koszulek: 3 × 79 zł = 237 zł. Następnie dodaję koszt dostawy: 237 zł + 12 zł = 249 zł.

Odpowiedź:
249 zł


In [50]:
# Sygnatura dla predyktora
optimized_solver_instruction_demos.predictors()[0]

Predict(StringSignature(problem -> reasoning, answer
    instructions='Przeczytaj uważnie podany problem matematyczny w języku polskim. Następnie rozwiąż go, przedstawiając szczegółowe rozumowanie krok po kroku, jasno wyjaśniając każdy etap obliczeń i założeń. Na końcu podaj ostateczną odpowiedź w formie liczby z odpowiednią jednostką (jeśli dotyczy), dbając o poprawność gramatyczną i zgodność z treścią zadania. Oddziel część z rozumowaniem od końcowej odpowiedzi.'
    problem = Field(annotation=str required=True json_schema_extra={'desc': 'Problem obliczeniowy do rozwiązania', '__dspy_field_type': 'input', 'prefix': 'Problem:'})
    reasoning = Field(annotation=str required=True json_schema_extra={'desc': 'Szczegółowe rozumowanie krok po kroku', '__dspy_field_type': 'output', 'prefix': 'Reasoning:'})
    answer = Field(annotation=str required=True json_schema_extra={'desc': "Końcowa odpowiedź na problem. Format to liczba z jednostką (w przypadku jednostek odmienialnych musi być podana

In [51]:
# Wybrane przykłady few-shot
optimized_solver_instruction_demos.predictors()[0].demos

[Example({'augmented': True, 'problem': 'Bilet do kina kosztuje 22 zł dla osoby dorosłej i 14 zł dla dziecka. Ile zapłaci rodzina składająca się z 2 dorosłych i 3 dzieci?', 'reasoning': 'Najpierw obliczam koszt biletów dla dorosłych: 2 osoby dorosłe × 22 zł = 44 zł.\nNastępnie obliczam koszt biletów dla dzieci: 3 dzieci × 14 zł = 42 zł.\nTeraz sumuję oba wyniki: 44 zł + 42 zł = 86 zł.', 'answer': '86 zł'}) (input_keys=None),
 Example({'problem': 'Jeśli 5 koszy zawiera po 8 jabłek każdy, to ile jest wszystkich jabłek?', 'reasoning': 'Aby obliczyć całkowitą liczbę jabłek, mnożę liczbę koszy przez liczbę jabłek w każdym koszu: 5 koszy × 8 jabłek = 40 jabłek.', 'answer': '40 jabłek'}) (input_keys={'problem'}),
 Example({'problem': 'Samochód przejechał 240 km w 3 godziny. Jaka była jego średnia prędkość?', 'reasoning': 'Aby obliczyć średnią prędkość, dzielę przebytą odległość przez czas: 240 km ÷ 3 h = 80 km/h.', 'answer': '80 km/h'}) (input_keys={'problem'}),
 Example({'problem': 'Na stole

In [52]:
# wyświetlmy przykładowy prompt
dspy.inspect_history(n=1)





[2025-05-01T21:08:52.645072]

System message:

Your input fields are:
1. `problem` (str): Problem obliczeniowy do rozwiązania
Your output fields are:
1. `reasoning` (str): Szczegółowe rozumowanie krok po kroku
2. `answer` (str): Końcowa odpowiedź na problem. Format to liczba z jednostką (w przypadku jednostek odmienialnych musi być podana w formie poprawnej gramatycznie, np. '2 kg jabłek', '5%' lub '8 zł').
All interactions will be structured in the following way, with the appropriate values filled in.

[[ ## problem ## ]]
{problem}

[[ ## reasoning ## ]]
{reasoning}

[[ ## answer ## ]]
{answer}

[[ ## completed ## ]]
In adhering to this structure, your objective is: 
        Przeczytaj uważnie podany problem matematyczny w języku polskim. Następnie rozwiąż go, przedstawiając szczegółowe rozumowanie krok po kroku, jasno wyjaśniając każdy etap obliczeń i założeń. Na końcu podaj ostateczną odpowiedź w formie liczby z odpowiednią jednostką (jeśli dotyczy), dbając o poprawność gramatyc

## Dodatkowe zasoby

- [Oficjalna dokumentacja DSPy](https://dspy-docs.vercel.app/)
- [Repozytorium GitHub](https://github.com/stanfordnlp/dspy)
- [Artykuł naukowy o DSPy](https://arxiv.org/abs/2310.03714)